# Virtualization and OpenShift Workflow Agent

## Overview

This notebook demonstrates how to build an AI-powered multi-agent workflow that interacts with **Virtual Machines (VMs)** and **OpenShift**. The agents process tasks such as listing VMs, retrieving VM details, and managing OpenShift migration plans using custom tools. 

By leveraging a machine learning model, the agents are able to process natural language instructions and execute specific tasks. The workflow is designed to be flexible, allowing different agents to collaborate through a shared state, with task data being persisted using **SQLite**.

## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [ ]:
%pip install langchain langgraph langgraph-checkpoint-sqlite langchain_community requests termcolor wikipedia arxiv

## 2. Configuring a Simple Model

In this section, we configure the machine learning model that we will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16").

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables us to perform model-based tasks using the provided configuration.

In [ ]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## 3. Defining the Shared State

In any multi-agent system, it’s crucial that agents can share information and coordinate their tasks. To achieve this, we use a **shared state** — a centralized data structure that agents read from and update as they perform their respective tasks. This shared state acts as a "memory" that persists between agent actions, enabling collaboration across different stages of the workflow.

In [ ]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any


class MultiAgentGraphState(TypedDict):
    input: str
    virt_engineer_response: Annotated[list, add_messages]
    ocp_engineer_response: Annotated[list, add_messages]

## 4. Setting Up Tools for the Workflow

In this section, we prepare the tools that will be used by the different agents in our workflow. Each agent has a specific set of tools they use to perform their tasks. Tools can represent any reusable function or utility that an agent can use to accomplish its assigned work.

We are introducing two sets of tools:

1. **OpenShift Tools**: Tools related to OpenShift operations, such as creating or starting a migration plan.
2. **vSphere Tools**: Tools for virtual machine operations, such as listing VMs or retrieving VM details.

### OpenShift Tools

The first set of tools we define is for OpenShift. These tools are used by the OpenShift engineer agent in the workflow. We utilize utility functions to fetch the tools' names and their descriptions, which will later be used in the workflow.

- **`create_migration_plan_tool`**: This tool is responsible for creating a migration plan in OpenShift.
- **`start_migration_plan_tool`**: This tool starts the migration plan that was created.

We then use helper functions (`get_tools_name` and `get_tools_description_json`) to get the names and descriptions of the tools, which will be helpful for dynamically assigning tasks to agents later on.


In [ ]:
import json
from utils.general.tools import (
    get_tools_name,
    get_tools_description_json,
)

from tools.openshift.migration_plan import (
    create_migration_plan_tool,
    start_migration_plan_tool,
)

ocp_tools = [create_migration_plan_tool, start_migration_plan_tool]

ocp_tools_name = get_tools_name(ocp_tools)
ocp_tools_description_json = get_tools_description_json(ocp_tools)

ocp_tools_description = json.dumps(ocp_tools_description_json, indent=4)

print(ocp_tools_description)

### vSphere Tools

Similarly, we define a set of tools for managing virtual machines. These tools will be used by the virtual engineer agent in the workflow.

- **`list_vms`**: This tool lists all the virtual machines available in the vSphere environment.
- **`retrieve_vm_details`**: This tool retrieves details about a specific virtual machine.

Like the OpenShift tools, we extract the names and descriptions of these tools for later use in the workflow.

In [ ]:
from tools.vsphere.vms import list_vms, retrieve_vm_details

virt_tools = [list_vms, retrieve_vm_details]

virt_tools_name = get_tools_name(virt_tools)
virt_tools_description_json = get_tools_description_json(virt_tools)

virt_tools_description = json.dumps(virt_tools_description_json, indent=4)

print(virt_tools_description)

## 5. Configuring Agent Nodes

Next, we will configure the virtual engineer and OpenShift engineer agents. These agents will be responsible for performing specific tasks in the workflow by utilizing the tools we just defined.

For each agent, we define:
- A **role** (e.g., `virt_engineer` or `ocp_engineer`).
- A set of **tools** that the agent can use to complete its tasks.
  
The agents will respond to the tasks dynamically, leveraging the tools to produce results.

In [ ]:
from agent.react_agent import ReactAgent

# Process a list of tasks for the Virtual Engineer.
def virt_eng_node_function(state: MultiAgentGraphState, tasks: list[str]):
    virt_agent = ReactAgent(
        state=state,
        role="virt_engineer",
        tools=virt_tools,
        ollama_service=ollama_service,
    )

    # Process each task and collect the responses
    responses = []
    for task in tasks:
        response = virt_agent.react(user_request=task)
        responses.append(response["virt_engineer_response"].content)

    return {"virt_engineer_response": responses}

# Process a list of tasks for the OpenShift Engineer.
def ocp_eng_node_function(state: MultiAgentGraphState, tasks: list[str]):
    ocp_agent = ReactAgent(
        state=state,
        role="ocp_engineer",
        tools=ocp_tools,
        ollama_service=ollama_service,
    )

    # Process each task and collect the responses
    responses = []
    for task in tasks:
        response = ocp_agent.react(user_request=task)
        responses.append(response["ocp_engineer_response"].content)

    return {"ocp_engineer_response": responses}

## 6. Workflow

In this section, we will finalize the workflow, integrate the agents, and run the graph. The workflow involves setting up virtual machine-related tasks for the **Virtual Engineer** and OpenShift-related tasks for the **OpenShift Engineer**. Both agents will receive lists of tasks and perform their respective operations using the tools provided. We'll use the `create_graph` function to set up the agent nodes, and `run_workflow` to execute the workflow.

In [ ]:
from state.state_graph import create_graph, run_workflow
from memory.sqlite_saver import initialize_memory

vm_name = "database-tttcp"
migration_plan_name = "migration-plan"

virt_tasks = [
    "This is a SINGLE task: List all VMs! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!",
    f"This is a SINGLE task: List the details for the vm named: {vm_name}! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!"
]

ocp_tasks = [
    f"You should execute only one task. THIS IS A SINGLE TASK: Use the create_migration_plan_tool to create a migration plan. Details: plan_name: {migration_plan_name}, vm_name:'{vm_name}'! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT START OR EXECUTE THE MIGRATION YET!",
    f"You should execute only one task. THIS IS A SINGLE TASK: Use the start_migration_plan_tool to start the {migration_plan_name}! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT CREATE THE MIGRATION!"
]


# Define a dictionary to map node names to their corresponding functions
agent_nodes = {
    "virt_engineer_node": lambda state: virt_eng_node_function(
        state=state, tasks=virt_tasks
    ),
    "ocp_engineer_node": lambda state: ocp_eng_node_function(
        state=state, tasks=ocp_tasks
    ),
}

memory = initialize_memory(":memory:")

# Step 2: Create the graph
graph = create_graph(
    MultiAgentGraphState,
    agent_nodes,
)

# Step 3: Define the input query and config
query = "Who's the USA's President?"
config = {"configurable": {"thread_id": "1"}}

# Step 4: Run the workflow
run_workflow(
    graph=graph,
    input_query=query,
    config=config,
    memory=memory,
    iterations=10,
    verbose=True,
)

<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To start a migration plan, we need to use the start_migration_plan_tool with the name of the migration plan and its corresponding namespace.",
    "action": "start_migration_plan_tool",
    "action_input": {
        "plan_name": "migration-plan-tttcp",
        "namespace": "openshift-mtv"
    }
}<|eot_id|>
<|python_tag|>start_migration_plan_tool.call({'plan_name': 'migration-plan-tttcp', 'namespace': 'openshift-mtv'})
<|eom_id|>


/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/4 Connected to Openshift...

<|start_header_id|>ipython<|end_header_id|>

Migration plan 'migration-plan-tttcp' not found.<|eot_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observation": "Migration plan 'migration-plan-tttcp' not found."
}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "answer": "Sorry, I cannot answer your query."
}<|eot_id|>
